In [12]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re 
import pandas as pd

In [13]:
executable_path = {'executable_path': 'C:\local\chromedriver_win32\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [15]:
# Retrieve page with the requests module
response = requests.get(url)

In [16]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [17]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [18]:
# results are returned as an iterable list
results = soup.find_all('li', class_="result-row")
#print(results)

In [21]:
title_results = soup.find_all("div", class_="content_title", limit=1)
#print(len(title_results))
news_title = title_results[0].text.strip()
print(news_title)

NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network


In [22]:
paragraph_results = soup.find_all("div", class_="rollover_description_inner", limit=1)
#print(len(paragraph_results))
news_p = paragraph_results[0].text.strip()
print(news_p)

Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.


In [23]:
# URL of page to be scraped
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

#browser = Browser('chrome')

#with Browser('chrome') as browser :
with browser as b :
    b.visit(jpl_url)
    button = b.find_by_id('full_image')
    #print(len(button))
    button.click()

    featured_image = b.find_by_xpath('//img[@class="fancybox-image"]')
    #print(len(featured_image))
    featured_image_url = featured_image["src"]
    print (featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA22574_ip.jpg


In [24]:
# URL of page to be scraped
twitter_url = "https://twitter.com/marswxreport?lang=en"
# Retrieve page with the requests module
twitter_response = requests.get(twitter_url)
# Create BeautifulSoup object; parse with 'html.parser'
twitter_soup = BeautifulSoup(twitter_response.text, 'html.parser')
#print(twitter_soup)

#twitter_results = twitter_soup.find_all("p", class_="TweetTextSize", limit=1)
#print(len(twitter_results))

#for r in twitter_results :
#    print(r.text.strip())
    
twitter_results = twitter_soup.find_all("p", string = re.compile('^InSight .*'), limit=1)
#print(len(twitter_results))
mars_weather = twitter_results[0].text.strip()
print(mars_weather)

InSight sol 436 (2020-02-17) low -94.4ºC (-137.9ºF) high -11.4ºC (11.4ºF)
winds from the SSE at 6.3 m/s (14.1 mph) gusting to 21.5 m/s (48.1 mph)
pressure at 6.30 hPa


In [25]:
# URL of page to be scraped
facts_url = "https://space-facts.com/mars/"
facts_df = pd.read_html(facts_url, attrs={'id': 'tablepress-p-mars'})[0]
facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [26]:
facts_table_html = facts_df.to_html()
facts_table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [28]:
ast_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
strip_word = "Enhanced"

#browser = Browser('chrome')

hemisphere_image_urls = []

with Browser('chrome', **executable_path) as browser :

    browser.visit(ast_url)
    links_found = browser.links.find_by_partial_href('Viking')
    #print(len(links_found))

    links_set = set()

    for l in links_found :
        #print(l["href"])
        links_set.add(l["href"])

    #print(len(links_set))

    for l in links_set :
        browser.visit(l)
        title_element = browser.find_by_xpath('//h2[@class="title"]')
        title_string = title_element.text.strip(strip_word).strip()
        print(title_string)
        img_sample_element = browser.find_by_text("Sample")
        print(img_sample_element["href"])
        tmp_dict = {"title" : title_string, "img_url" : img_sample_element["href"]}
        hemisphere_image_urls.append(tmp_dict)

hemisphere_image_urls

Valles Marineris Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
Cerberus Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg


[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}]